# <a id='toc1_'></a>[PCE:Couplage de Rdkit Crest-xTB avec AlGORITHME DE SCHARBER ](#toc0_)

1. **MVOTO KONGO Patrick Sorrel**, sorrel.mvoto@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I 
Etudiant de Master au Laboratoire de 
    * Physique Atomique Moleculaire et Biophysique

JUIN 2024

* Nos travaux de mémoire sont basés sur le flux de travail de Tartarus. L'algorithme est le suivant :
<img src="./Tartarsus.png" width="1500"></center>

 Une illustration du cadre Tartarus, mettant en évidence les tâches de conception réelles qui sont définies et associées à des flux de travail et des ensembles de données de simulation dans Tartarus




*  Notre objectif de recherche consiste à concevoir des petites molécules dotées de propriétés électroniques spécifiques, notamment des molécules capables d'effectuer la séparation des charges, inspirées de la conception photovoltaïque organique (OPV). Nous avons deux tâches individuelles :

- Recherche d'une molécule donneuse organique à utiliser avec l'ester méthylique de l'acide l'ester méthylique de
    l'acide ${[6,6]­phényl­C61­butyrique(PCBM)}$.
- Recherche d'une molécule accepteuse à utiliser dans des dispositifs basés sur le poly[N­90­heptadécanyl­2,7­carbazole­alt­5,5­(40,70­di­2­thiényl­20,10,30­benzothiaMachine  (PCDTBT)).

*  Notre travail actuel porte sur le modèle ci-dessous :
<img src="./opv.png" width="2000"></center>

 schématique du flux de travail de simulation de propriétés pour la conception de références photovoltaïques organiques

In [5]:
import pygments
import pandas as pd

#      Modele de sharber
####  <a id='toc1_3_'></a>[Utilisation de crest et xTB pour la recherches des conformers](#toc0_)
 <center> <img src = "./crest.jpeg" width = "600">
 <img src = "./xtb.jpeg" width = "600"> </center> 

    dtb = []
    yesso = ["GAP_xtb(eV)","MO_DIP_xtb (eV)","HOMO_xtb(eV)","LUMO_xtb(eV)"]

# Create a copy to avoid modifying the original DataFrame

    for i in range(1,19425):
  
        # Read the output
        with open(f'qm9_dump_{i}', 'r') as f:
            text_content = f.readlines()
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        text_content = text_content[output_index[0]:]
        homo_data = [x for x in text_content if '(HOMO)' in x]
        lumo_data = [x for x in text_content if '(LUMO)' in x]
        homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
        mol_dipole = [text_content[i:i+4] for i, x in enumerate(text_content) if 'molecular dipole:' in x]
        lumo_val = float(lumo_data[0].split(' ')[-2])
        homo_val = float(homo_data[0].split(' ')[-2])
        homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])
        mol_dipole_val = float(mol_dipole[0][-1].split(' ')[-1])



        # Write the properties to a single file (modify as needed)
        with open(os.path.join(directory, 'properties.txt'), 'a') as f:
            f.write(f'SMILES: {smiles}\n')
            f.write(f'LUMO: {lumo_val}\n')
            f.write(f'HOMO: {homo_val}\n')
            f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')
        dtb.append([homo_lumo_val, mol_dipole_val, homo_val, lumo_val])
        os.chdir('..')

    df_xtb = pd.DataFrame(dtb, columns=yesso)

In [69]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from pyscf.data import nist

au2ev = nist.HARTREE2EV
yesso = [ "GAP_xtb(eV)"]
dtb1 = []
# Remarque : Assurez-vous que 'directory' est défini et accessible
directory = 'GDB9_1'  
# Paramètres
input_dir = 'GDB-9'  # Dossier contenant les fichiers .xyz d'origine
output_dir = 'GDB-V2024'  # Dossier où seront créés les nouveaux fichiers .xyz
output_file = 'Patrick_GDB-9_dataframe.csv'  # Nom du fichier CSV pour le dataframe
max_files = 20885  # Nombre maximum de fichiers à traiter
# Créer le dossier de sortie s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
def extract_value_from_line(line, pattern):
    match = re.search(pattern, line)
    if match:
        return float(match.group(1))
    return None
# Fonction pour nettoyer les fichiers XYZ
def cleanup_GDB9_xyz(fname):
    ind = open(fname).readlines()
    nAts = int(ind[0])
    smi = ind[-2].split()[-1]
    ind[1] = '\n'
    ind = ind[:nAts+2]
    for i in range(2, nAts+2):
        l = ind[i]
        l = l.split()
        l.pop(-1)
        ind[i] = '\t'.join(l) + '\n'
    ind = ''.join(ind)
    return ind, smi 

# Initialiser la liste pour le dataframe
dtb = []

# Parcourir les 1000 premiers fichiers .xyz d'origine
for i in range(1, max_files + 1):
    filename = f'dsgdb9nsd_{i:06d}.xyz'
    filepath = os.path.join(input_dir, filename)

    if os.path.exists(filepath):
        # Construire le nouveau nom de fichier
        new_filename = f'GDB{i}.xyz'
        new_filepath = os.path.join(output_dir, new_filename)

        # Extraire les coordonnées XYZ, le nom du monde GDB, et les propriétés
        with open(filepath, 'r') as f:
            lines = f.readlines()
            num_atoms = int(lines[0].strip())
            world_name = lines[1].strip().split('\t')[0]
            properties = [float(x) for x in lines[1].strip().split('\t')[1:]]
            xyz_coords = [line.strip().split() for line in lines[2:2+num_atoms]]
            smiles = lines[-2].strip().split()[-2]
            smiles_12 = lines[-2].strip().split()[-1]

        # Utiliser la fonction pour nettoyer et obtenir les coordonnées XYZ
        ind, smi = cleanup_GDB9_xyz(filepath)

        # Écrire les nouvelles coordonnées XYZ dans le nouveau fichier
        with open(new_filepath, 'w') as f:
            f.write(f'{num_atoms}\n')
            f.write('\n')
            for row in xyz_coords:
                formatted_row = '{:<2} {:>15} {:>15} {:>15}\n'.format(row[0], row[1], row[2], row[3])
                f.write(formatted_row)
            mol = Chem.MolFromSmiles(smiles)
            homo = properties[5]*au2ev 
            lumo = properties[6]*au2ev 
            gap = properties[7]*au2ev 
              
    filepath = f'qm9_dump{i}'
    filename = os.path.join(directory, filepath)
    try:
        # Lire le fichier de sortie
        with open(filename, 'r') as f:
            text_content = f.readlines()

        # Rechercher l'index de la section de sortie pertinente
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        if not output_index:
            continue  # Si 'Property Printout' n'est pas trouvé, passer au fichier suivant

        text_content = text_content[output_index[0]:]

        # Extraire les données pertinentes
        homo_lumo_val = None
        mol_dipole_val = None

        for line in text_content:
            if '(HOMO)' in line:
                homo_val = extract_value_from_line(line, r'([-\d\.]+) eV')
            if '(LUMO)' in line:
                lumo_val = extract_value_from_line(line, r'([-\d\.]+) eV')
            if 'HOMO-LUMO GAP' in line:
                homo_lumo_val = extract_value_from_line(line, r'([-\d\.]+) eV')

 

        # Ajouter les données à la liste
        dtb.append([homo_lumo_val])

    except FileNotFoundError:
        print(f"Le fichier {filename} n'a pas été trouvé. Passage au fichier suivant.")
    except Exception as e:
        print(f"Erreur lors du traitement du fichier {filename}: {e}")

# Créer le DataFrame à partir des données collectées
df_xtb = pd.DataFrame(dtb, columns=yesso)


Le fichier GDB9_1/qm9_dump58 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump61 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump80 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump185 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump186 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump245 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump267 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump280 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump286 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump752 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump753 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump754 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump755 n'a pas été trouvé. Passage au fichie

In [70]:
df_xtb

,GAP_xtb(eV)
0,17.353095
1,12.232824
2,14.386316
3,7.287776
4,5.949769
...,...
19420,9.373166
19421,8.192813
19422,8.649156
19423,8.613450


In [73]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from pyscf.data import nist

au2ev = nist.HARTREE2EV
yesso = ["smiles_key","smiles","mol","HOMO(eV)","LUMO(eV)","GAP(eV)", "HOMO_xtb(eV)", "LUMO_xtb(eV)"]
dtb1 = []
# Remarque : Assurez-vous que 'directory' est défini et accessible
directory = 'GDB9_1'  
# Paramètres
input_dir = 'GDB-9'  # Dossier contenant les fichiers .xyz d'origine
output_dir = 'GDB-V2024'  # Dossier où seront créés les nouveaux fichiers .xyz
output_file = 'Patrick_GDB-9_dataframe.csv'  # Nom du fichier CSV pour le dataframe
max_files = 20885  # Nombre maximum de fichiers à traiter
# Créer le dossier de sortie s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Fonction pour nettoyer les fichiers XYZ
def cleanup_GDB9_xyz(fname):
    ind = open(fname).readlines()
    nAts = int(ind[0])
    smi = ind[-2].split()[-1]
    ind[1] = '\n'
    ind = ind[:nAts+2]
    for i in range(2, nAts+2):
        l = ind[i]
        l = l.split()
        l.pop(-1)
        ind[i] = '\t'.join(l) + '\n'
    ind = ''.join(ind)
    return ind, smi 

# Initialiser la liste pour le dataframe
data = []

# Parcourir les 1000 premiers fichiers .xyz d'origine
for i in range(1, max_files + 1):
    filename = f'dsgdb9nsd_{i:06d}.xyz'
    filepath = os.path.join(input_dir, filename)

    if os.path.exists(filepath):
        # Construire le nouveau nom de fichier
        new_filename = f'GDB{i}.xyz'
        new_filepath = os.path.join(output_dir, new_filename)

        # Extraire les coordonnées XYZ, le nom du monde GDB, et les propriétés
        with open(filepath, 'r') as f:
            lines = f.readlines()
            num_atoms = int(lines[0].strip())
            world_name = lines[1].strip().split('\t')[0]
            properties = [float(x) for x in lines[1].strip().split('\t')[1:]]
            xyz_coords = [line.strip().split() for line in lines[2:2+num_atoms]]
            smiles = lines[-2].strip().split()[-2]
            smiles_12 = lines[-2].strip().split()[-1]

        # Utiliser la fonction pour nettoyer et obtenir les coordonnées XYZ
        ind, smi = cleanup_GDB9_xyz(filepath)

        # Écrire les nouvelles coordonnées XYZ dans le nouveau fichier
        with open(new_filepath, 'w') as f:
            f.write(f'{num_atoms}\n')
            f.write('\n')
            for row in xyz_coords:
                formatted_row = '{:<2} {:>15} {:>15} {:>15}\n'.format(row[0], row[1], row[2], row[3])
                f.write(formatted_row)
            mol = Chem.MolFromSmiles(smiles)
            homo = properties[5]*au2ev 
            lumo = properties[6]*au2ev 
            gap = properties[7]*au2ev 
              
    filepath = f'qm9_dump{i}'
    filepath1 = f' gdb {i}'
   
    filename= os.path.join(directory, filepath)
    try:
        # Lire le fichier de sortie
        with open(filename, 'r') as f:
            text_content = f.readlines()

        # Rechercher l'index de la section de sortie pertinente
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        if not output_index:
            continue  # Si 'Property Printout' n'est pas trouvé, passer au fichier suivan
        text_content = text_content[output_index[0]:]
        # Extraire les données pertinentes
        homo_data = [x for x in text_content if '(HOMO)' in x]
        lumo_data = [x for x in text_content if '(LUMO)' in x]
 

        # Extraire les valeurs numériques
        lumo_val = float(lumo_data[0].split()[-2])
        homo_val = float(homo_data[0].split()[-2])


        # Écrire les propriétés dans un fichier unique (modifiez si nécessaire)
        with open(os.path.join(directory, 'properties.txt'), 'a') as f:
            f.write(f'Fichier: {filepath}\n')
            f.write(f'LUMO: {lumo_val}\n')

        # Ajouter les données à la liste
        dtb1.append([world_name,smiles, mol,homo,lumo,gap, homo_val, lumo_val])

    except FileNotFoundError:
        print(f"Le fichier {filename} n'a pas été trouvé. Passage au fichier suivant.")
    except Exception as e:
        print(f"Erreur lors du traitement du fichier {filename}: {e}")

# Créer le DataFrame à partir des données collectées
df_xtb1 = pd.DataFrame(dtb1, columns=yesso)


Le fichier GDB9_1/qm9_dump58 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump61 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump80 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump185 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump186 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump245 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump267 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump280 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump286 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump752 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump753 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump754 n'a pas été trouvé. Passage au fichier suivant.
Le fichier GDB9_1/qm9_dump755 n'a pas été trouvé. Passage au fichie

In [74]:
df_xtb1

,smiles_key,smiles,mol,HOMO(eV),LUMO(eV),GAP(eV),HOMO_xtb(eV),LUMO_xtb(eV)
0,gdb 1,C,<rdkit.Chem.rdchem.Mol object at 0x788d81a32e40>,-10.549854,3.186453,13.736308,-12.7202,4.6329
1,gdb 2,N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32dd0>,-6.993326,2.255824,9.249150,-10.5031,1.7297
2,gdb 3,O,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bc10>,-7.967494,1.869422,9.836916,-12.1467,2.2396
3,gdb 4,C#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bba0>,-7.741639,1.376896,9.118535,-11.5117,-4.2239
4,gdb 5,C#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32c80>,-9.806984,0.519737,10.329442,-12.2419,-6.2921
...,...,...,...,...,...,...,...,...
19420,gdb 19752,CC1C2C3CC1CC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156ff90>,-7.028701,2.517053,9.545754,-10.7227,-1.3496
19421,gdb 19753,CC1C2C3CC1CN23,<rdkit.Chem.rdchem.Mol object at 0x788d8156fa50>,-6.334811,2.402765,8.737576,-10.0806,-1.8878
19422,gdb 19754,CC1C2C3CC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f270>,-6.190590,2.530659,8.721249,-10.3509,-1.7017
19423,gdb 19755,CC1C2C3OC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f040>,-6.498079,2.473515,8.971594,-10.6935,-2.0800


 -12.7202 (HOMO)
         5                          0.1702568               4.6329 (LUMO)
          17.353095283966 eV

In [75]:
df_dFT=pd.concat([df_xtb1,df_xtb ], axis=1)
df_dFT

,smiles_key,smiles,mol,HOMO(eV),LUMO(eV),GAP(eV),HOMO_xtb(eV),LUMO_xtb(eV),GAP_xtb(eV)
0,gdb 1,C,<rdkit.Chem.rdchem.Mol object at 0x788d81a32e40>,-10.549854,3.186453,13.736308,-12.7202,4.6329,17.353095
1,gdb 2,N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32dd0>,-6.993326,2.255824,9.249150,-10.5031,1.7297,12.232824
2,gdb 3,O,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bc10>,-7.967494,1.869422,9.836916,-12.1467,2.2396,14.386316
3,gdb 4,C#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bba0>,-7.741639,1.376896,9.118535,-11.5117,-4.2239,7.287776
4,gdb 5,C#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32c80>,-9.806984,0.519737,10.329442,-12.2419,-6.2921,5.949769
...,...,...,...,...,...,...,...,...,...
19420,gdb 19752,CC1C2C3CC1CC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156ff90>,-7.028701,2.517053,9.545754,-10.7227,-1.3496,9.373166
19421,gdb 19753,CC1C2C3CC1CN23,<rdkit.Chem.rdchem.Mol object at 0x788d8156fa50>,-6.334811,2.402765,8.737576,-10.0806,-1.8878,8.192813
19422,gdb 19754,CC1C2C3CC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f270>,-6.190590,2.530659,8.721249,-10.3509,-1.7017,8.649156
19423,gdb 19755,CC1C2C3OC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f040>,-6.498079,2.473515,8.971594,-10.6935,-2.0800,8.613450


In [76]:
df_dFT.to_csv('XTB_CREST_GDB9.csv', index=False)  

In [77]:

#counter = 0
df1=df_dFT[(df_dFT['HOMO_xtb(eV)']< 0) &(df_dFT['LUMO_xtb(eV)']<0)]
# # Utiliser une boucle for pour filtrer les données et les ajouter à la nouvelle DataFrame
# for i in range(len(df1)):
#     df=df1.loc[i, 'HOMO(eV)'] < 0 and df1.loc[i, 'LUMO(eV)'] < 0
#     df.loc[counter] = df1.loc[i]
#     counter += 1

# # Couper la DataFrame à la taille réelle des données filtrées
# df = df[:counter]

# # Afficher la nouvelle DataFrame filtrée
# print(df)
df1.to_csv('my_XTB_CREST_GDB9.csv', index=False)

In [78]:
df1

,smiles_key,smiles,mol,HOMO(eV),LUMO(eV),GAP(eV),HOMO_xtb(eV),LUMO_xtb(eV),GAP_xtb(eV)
3,gdb 4,C#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bba0>,-7.741639,1.376896,9.118535,-11.5117,-4.2239,7.287776
4,gdb 5,C#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32c80>,-9.806984,0.519737,10.329442,-12.2419,-6.2921,5.949769
5,gdb 6,C=O,<rdkit.Chem.rdchem.Mol object at 0x788d81a17b30>,-7.265440,-1.104782,6.157937,-11.5341,-7.9137,3.620361
8,gdb 9,CC#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a259e0>,-7.099451,1.668058,8.767509,-10.8802,-3.8011,7.079141
9,gdb 10,CC#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a25ba0>,-8.881796,1.023148,9.904945,-11.6114,-5.5012,6.110195
...,...,...,...,...,...,...,...,...,...
19420,gdb 19752,CC1C2C3CC1CC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156ff90>,-7.028701,2.517053,9.545754,-10.7227,-1.3496,9.373166
19421,gdb 19753,CC1C2C3CC1CN23,<rdkit.Chem.rdchem.Mol object at 0x788d8156fa50>,-6.334811,2.402765,8.737576,-10.0806,-1.8878,8.192813
19422,gdb 19754,CC1C2C3CC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f270>,-6.190590,2.530659,8.721249,-10.3509,-1.7017,8.649156
19423,gdb 19755,CC1C2C3OC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f040>,-6.498079,2.473515,8.971594,-10.6935,-2.0800,8.613450
